In [ ]:
#Classification

#================================================================== Data Split ======================================================

import numpy as np
import random, shutil, glob

num_test_file = 100

File_List = glob.glob('file path_total')
list_index = list(np.arange(0, len(File_List), 1))
random_index = random.sample(list_index,len(File_List))
for i in range(len(random_index)):
    random_index_ = random_index[i]

#=================================================================== Test data set =====================================================    
    if i < num_test_file:
        shutil.copy('file path_total' %random_index_, 'file path_Test' %random_index_)
#============================================================ Training data set + validation data set ==========================================        
    if num_test_file <= i:
        shutil.copy('file path_total' %random_index_, 'file path_Training' %random_index_)

In [ ]:
#Classification

import tensorflow as tf
import numpy as np
import time, glob

tf.set_random_seed(777)

learning_rate = 0.001
num_training_data = 69943
num_validation_data = 10000
num_test_data = 10000
X_length = 4012
batch_size_ = 1000
n_classes = 21
epochs = 1
before_gen_num= 0

#----------------------------------------------------------------------------------------------------------------

graph = tf.Graph()
with graph.as_default():
    
    input_data = tf.placeholder(tf.float32, shape=[None, X_length])
    input_data_vld = tf.placeholder(tf.float32, shape=[None, X_length])
    input_data_tes = tf.placeholder(tf.float32, shape=[None, X_length])
    
    dataset = tf.data.Dataset.from_tensor_slices(input_data).shuffle(num_training_data).repeat().batch(batch_size_)
    iterator = tf.compat.v1.data.make_initializable_iterator(dataset)
    xy_data = iterator.get_next()
    
    y_1=tf.cast(xy_data[:,-8], tf.int32)
    y_2=tf.cast(xy_data[:,-7], tf.int32)
    y_3=tf.cast(xy_data[:,-6], tf.int32)

    y_1=tf.one_hot(y_1, n_classes)
    y_2=tf.one_hot(y_2, n_classes)
    y_3=tf.one_hot(y_3, n_classes)
    y_data = tf.add_n([y_1, y_2, y_3])
    y_data_ = y_1*tf.reshape(xy_data[:,-5],[-1,1])+y_2*tf.reshape(xy_data[:,-4],[-1,1])+y_3*tf.reshape(xy_data[:,-3],[-1,1])
    y_data = tf.cast(y_data, tf.float32)
    y_data_ = tf.cast(y_data_, tf.float32)
    
    X_train, y_train, y_train_, y_train_p, y_train_ind = [xy_data[:,:-11], y_data, y_data_, xy_data[:,-5:-2], xy_data[:,-1:]]
    
    print(X_train, y_train, y_train_, y_train_p, y_train_ind)
    
#========================================================================== Validation set ==========================================   
    
    dataset_vld = tf.data.Dataset.from_tensor_slices(input_data_vld).shuffle(num_validation_data).repeat().batch(batch_size_)
    iterator_vld = tf.compat.v1.data.make_initializable_iterator(dataset_vld)
    xy_data_vld = iterator_vld.get_next()
    
    y_1_vld=tf.cast(xy_data_vld[:,-8], tf.int32)
    y_2_vld=tf.cast(xy_data_vld[:,-7], tf.int32)
    y_3_vld=tf.cast(xy_data_vld[:,-6], tf.int32)

    y_1_vld=tf.one_hot(y_1_vld, n_classes)
    y_2_vld=tf.one_hot(y_2_vld, n_classes)
    y_3_vld=tf.one_hot(y_3_vld, n_classes)
    y_data_vld = tf.add_n([y_1_vld, y_2_vld, y_3_vld])
    y_data_vld_ = y_1_vld*tf.reshape(xy_data_vld[:,-5],[-1,1])+y_2_vld*tf.reshape(xy_data_vld[:,-4],[-1,1])+y_3_vld*tf.reshape(xy_data_vld[:,-3],[-1,1])
    y_data_vld = tf.cast(y_data_vld, tf.float32)
    y_data_vld_ = tf.cast(y_data_vld_, tf.float32)
    
    X_vld, y_vld, y_vld_, y_vld_p, y_vld_ind = [xy_data_vld[:,:-11], y_data_vld, y_data_vld_, xy_data_vld[:,-5:-2], xy_data_vld[:,-1:]]
        
#========================================================================== Test set ==========================================     
    
    dataset_tes = tf.data.Dataset.from_tensor_slices(input_data_tes).shuffle(num_test_data).repeat().batch(batch_size_)
    iterator_tes = tf.compat.v1.data.make_initializable_iterator(dataset_tes)
    xy_data_tes = iterator_tes.get_next()
    
    y_1_tes=tf.cast(xy_data_tes[:,-8], tf.int32)
    y_2_tes=tf.cast(xy_data_tes[:,-7], tf.int32)
    y_3_tes=tf.cast(xy_data_tes[:,-6], tf.int32)

    y_1_tes=tf.one_hot(y_1_tes, n_classes)
    y_2_tes=tf.one_hot(y_2_tes, n_classes)
    y_3_tes=tf.one_hot(y_3_tes, n_classes)
    y_data_tes = tf.add_n([y_1_tes, y_2_tes, y_3_tes])
    y_data_tes_ = y_1_tes*tf.reshape(xy_data_tes[:,-5],[-1,1])+y_2_tes*tf.reshape(xy_data_tes[:,-4],[-1,1])+y_3_tes*tf.reshape(xy_data_tes[:,-3],[-1,1])
    y_data_tes = tf.cast(y_data_tes, tf.float32)
    y_data_tes_ = tf.cast(y_data_tes_, tf.float32)
    
    X_tes, y_tes, y_tes_, y_tes_p, y_tes_ind = [xy_data_tes[:,:-11], y_data_tes, y_data_tes_, xy_data_tes[:,-5:-2], xy_data_tes[:,-1:]]

In [ ]:
#Classification

with graph.as_default():
    
    inputs_ = tf.placeholder(tf.float32, [None, 4001, 1], name = 'inputs')
    labels_1 = tf.placeholder(tf.float32, [None, n_classes], name = 'labels_1')
    logit_num = tf.placeholder(tf.int32, [None, 3], name = 'logits_Top_3')
    label_num = tf.placeholder(tf.int32, [None, 3], name = 'labels_Top_3')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

    #   model architecture(CNN_2)

    '''
    
    conv1 = tf.keras.layers.Conv1D(filters=64, kernel_size=15, strides=2,padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(inputs_)
    max_pool_1 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=64, kernel_size=10, strides=3, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_1)
    max_pool_2 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=3, padding='same')(conv2)
    flat = tf.keras.layers.Flatten()(max_pool_2)
    flat = tf.keras.layers.Dropout(rate=keep_prob_)(flat)
    logits = tf.keras.layers.Dense(units=2000, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(flat)
    logits= tf.keras.layers.Dropout(rate=keep_prob_)(logits)
    logits_ = tf.keras.layers.Dense(units=500, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits)
    logits_= tf.keras.layers.Dropout(rate=keep_prob_)(logits_)
    logits_2 = tf.keras.layers.Dense(units=n_classes, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits_)
    
    '''
    
    
    #   model architecture(CNN_3)
    
    '''
    
    conv1 = tf.keras.layers.Conv1D(filters=64, kernel_size=20, strides=1,padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(inputs_)
    max_pool_1 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=3, padding='same')(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=64, kernel_size=15, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_1)
    max_pool_2 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=3, padding='same')(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=64, kernel_size=10, strides=2, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_2)
    max_pool_3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv3)
    flat = tf.keras.layers.Flatten()(max_pool_3)
    flat = tf.keras.layers.Dropout(rate=keep_prob_)(flat)
    logits = tf.keras.layers.Dense(units=2500, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(flat)
    logits= tf.keras.layers.Dropout(rate=keep_prob_)(logits)
    logits_ = tf.keras.layers.Dense(units=1000, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits)
    logits_= tf.keras.layers.Dropout(rate=keep_prob_)(logits_)
    logits_2 = tf.keras.layers.Dense(units=n_classes, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits_)
    
    '''
    #   model architecture(CNN_4)
    
    '''
    
    conv1 = tf.keras.layers.Conv1D(filters=64, kernel_size=25, strides=1,padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(inputs_)
    max_pool_1 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=64, kernel_size=20, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_1)
    max_pool_2 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=64, kernel_size=15, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_2)
    max_pool_3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv3)
    conv4 = tf.keras.layers.Conv1D(filters=64, kernel_size=10, strides=2,padding='same', 
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_3)
    max_pool_4 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv4) 
    flat = tf.keras.layers.Flatten()(max_pool_4)
    flat = tf.keras.layers.Dropout(rate=keep_prob_)(flat)
    logits = tf.keras.layers.Dense(units=2500, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(flat)
    logits= tf.keras.layers.Dropout(rate=keep_prob_)(logits)
    logits_ = tf.keras.layers.Dense(units=1000, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits)
    logits_= tf.keras.layers.Dropout(rate=keep_prob_)(logits_)
    logits_2 = tf.keras.layers.Dense(units=n_classes, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits_)
    
    '''
    
    #   model architecture(CNN_5)
    
    '''
    
    conv1 = tf.keras.layers.Conv1D(filters=64, kernel_size=30, strides=1,padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(inputs_)
    max_pool_1 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=64, kernel_size=25, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_1)
    max_pool_2 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=64, kernel_size=20, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_2)
    max_pool_3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv3)
    conv4 = tf.keras.layers.Conv1D(filters=64, kernel_size=15, strides=1,padding='same', 
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_3)
    max_pool_4 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv4) 
    conv5 = tf.keras.layers.Conv1D(filters=64, kernel_size=10, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_4)
    max_pool_5 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv5)
    flat = tf.keras.layers.Flatten()(max_pool_5)
    flat = tf.keras.layers.Dropout(rate=keep_prob_)(flat)
    logits = tf.keras.layers.Dense(units=2500, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(flat)
    logits= tf.keras.layers.Dropout(rate=keep_prob_)(logits)
    logits_ = tf.keras.layers.Dense(units=1000, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits)
    logits_= tf.keras.layers.Dropout(rate=keep_prob_)(logits_)
    logits_2 = tf.keras.layers.Dense(units=n_classes, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits_)
    
    '''
    
    #   model architecture(CNN_6)
    
    
    conv1 = tf.keras.layers.Conv1D(filters=64, kernel_size=35, strides=1,padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(inputs_)
    max_pool_1 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=64, kernel_size=30, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_1)
    max_pool_2 = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=64, kernel_size=25, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_2)
    max_pool_3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv3)
    conv4 = tf.keras.layers.Conv1D(filters=64, kernel_size=20, strides=1,padding='same', 
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_3)
    max_pool_4 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv4) 
    conv5 = tf.keras.layers.Conv1D(filters=64, kernel_size=15, strides=1, padding='same',
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_4)
    max_pool_5 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv5)
    conv6 = tf.keras.layers.Conv1D(filters=64, kernel_size=10, strides=1,padding='same', 
                                   kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform(), activation = tf.nn.relu)(max_pool_5)
    max_pool_6 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='same')(conv6) 
    flat = tf.keras.layers.Flatten()(max_pool_6)
    flat = tf.keras.layers.Dropout(rate=keep_prob_)(flat)
    logits = tf.keras.layers.Dense(units=2500, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(flat)
    logits= tf.keras.layers.Dropout(rate=keep_prob_)(logits)
    logits_ = tf.keras.layers.Dense(units=1000, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits)
    logits_= tf.keras.layers.Dropout(rate=keep_prob_)(logits_)
    logits_2 = tf.keras.layers.Dense(units=n_classes, kernel_initializer=tf.compat.v1.keras.initializers.glorot_uniform())(logits_)
    
    
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_2, labels=labels_1))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    correct_pred = tf.math.equal(logit_num, label_num)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [ ]:
with graph.as_default():
    saver = tf.train.Saver()
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        
        train_result_list=[]
        train_result_list_1=[]
        
        validation_result_list=[]
        validation_result_list_1=[]
        file_list_total= glob.glob('/file path/*.csv')
        print(file_list_total)
        arrays = [np.loadtxt(file_name, delimiter=',', dtype=np.float32) for file_name in file_list_total]
        stack = np.concatenate(arrays, axis=0)
        np.random.shuffle(stack)
        data = stack[:-num_validation_data]
        data_vld = stack[-num_validation_data:]
                
        sess.run(iterator.initializer, feed_dict = {input_data:data})
        sess.run(iterator_vld.initializer, feed_dict = {input_data_vld:data_vld})    
        
        iteration = 1
        
        if e+before_gen_num ==0:
            sess.run(tf.global_variables_initializer())
            
        else:
            saver.restore(sess, 'file path')

        for i in range(70):     
            sum_ = 0
            sum_1 = 0
            X_tr, y_tr, y_tr_, y_tr_p, y_ind = sess.run([X_train, y_train, y_train_, y_train_p, y_train_ind])
            X_tr= np.reshape(X_tr, (-1, 4001, 1))
            feed = {inputs_ : X_tr, labels_1 : y_tr, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            loss, _ , logit = sess.run([cost, optimizer, logits_2], feed_dict = feed)
            y_lab = np.empty([batch_size_, 3])
            y_logit = np.empty([batch_size_, 3])
            
            for i in range(batch_size_):
                if y_ind[i,0] == 2:
                    y_lab[i]=np.argsort(y_tr_[i])[-3:]
                    y_lab[i]=np.sort(y_lab[i])
                    y_logit[i]=np.argsort(logit[i])[-3:]
                    y_logit[i]=np.sort(y_logit[i])
                    b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                    sum_1= sum_1+b1
                    if b1 == 3:
                        sum_+=1
                elif y_ind[i,0] == 1:
                    z=np.argsort(y_tr_[i])[-2:]
                    y_lab[i]=np.append(z, [-1])
                    y_lab[i]=np.sort(y_lab[i])
                    z_=np.argsort(logit[i])[-2:]
                    y_logit[i]=np.append(z_, [-1])
                    y_logit[i]=np.sort(y_logit[i])
                    b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                    sum_1 = sum_1+b1
                    if b1 == 3:
                        sum_+=1         
                elif y_ind[i,0] == 0:
                    z=np.argsort(y_tr_[i])[-1:]
                    y_lab[i]=np.append(z, [-1,-2])
                    y_lab[i]=np.sort(y_lab[i])
                    z_=np.argsort(logit[i])[-1:]
                    y_logit[i]=np.append(z_, [-1,-2])
                    y_logit[i]=np.sort(y_logit[i])
                    b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                    sum_1 = sum_1+b1
                    if b1 == 3:
                        sum_+=1
            acc_=sum_/len(y_lab)
            acc_1=sum_1/(len(y_lab)*y_lab.shape[1])
            
            result=['Epoch:' , e, 'iteration:', iteration, 'Train_loss:', loss, 'Train_sample_acc:', acc_]
            result_1=['Epoch:' , e, 'iteration:', iteration, 'Train_loss:', loss, 'Train_constituent_acc:', acc_1]
            
            train_result_list.append(result)
            train_result_list_1.append(result_1)
            
            print("Epoch: {}/{}".format(e, epochs),
                   "Iteration: {:d}".format(iteration),
                   "Train loss: {:6f}".format(loss),
                   "Train sample acc: {:.6f}".format(acc_),
                   "Train constituent acc_1: {:.6f}".format(acc_1))

###================================================================ VALIDATION =====================================            
            
            if (iteration %5 == 0):
                sum_ = 0
                sum_1 = 0
                X_vd, y_vd, y_vd_, y_vd_p, y_ind_vd = sess.run([X_vld, y_vld, y_vld_, y_vld_p, y_vld_ind])
                X_vd= np.reshape(X_vd, (-1, 4001, 1))
                feed = {inputs_ : X_vd, labels_1 : y_vd, keep_prob_ : 1.0, learning_rate_ : learning_rate}
                loss_vd,  logit_vd = sess.run([cost, logits_2], feed_dict = feed)
                y_lab_vd = np.empty([batch_size_, 3])
                y_logit_vd = np.empty([batch_size_, 3])
                for i in range(batch_size_):
                    if y_ind_vd[i,0] == 2:
                        y_lab_vd[i]=np.argsort(y_vd_[i])[-3:]
                        y_lab_vd[i]=np.sort(y_lab_vd[i])
                        y_logit_vd[i]=np.argsort(logit_vd[i])[-3:]
                        y_logit_vd[i]=np.sort(y_logit_vd[i])
                        b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                        sum_1= sum_1+b1
                        if b1 == 3:
                            sum_+=1
                    elif y_ind_vd[i,0] == 1:
                        z=np.argsort(y_vd_[i])[-2:]
                        y_lab_vd[i]=np.append(z, [-1])
                        y_lab_vd[i]=np.sort(y_lab_vd[i])                    
                        z_=np.argsort(logit_vd[i])[-2:]
                        y_logit_vd[i]=np.append(z_, [-1])
                        y_logit_vd[i]=np.sort(y_logit_vd[i])
                        b1 = len(list(set(y_lab_vd[i]).intersection(y_logit_vd[i])))
                        sum_1= sum_1+b1
                        if b1 == 3:
                            sum_+=1
                    elif y_ind_vd[i,0] == 0:
                        z=np.argsort(y_vd_[i])[-1:]
                        y_lab_vd[i]=np.append(z, [-1,-2])
                        y_lab_vd[i]=np.sort(y_lab_vd[i])
                        z_=np.argsort(logit_vd[i])[-1:]
                        y_logit_vd[i]=np.append(z_, [-1,-2])
                        y_logit_vd[i]=np.sort(y_logit_vd[i])
                        b1 = len(list(set(y_lab_vd[i]).intersection(y_logit_vd[i])))
                        sum_1= sum_1+b1
                        if b1 == 3:
                            sum_+=1
                acc_vd_=sum_/len(y_lab_vd)            
                acc_vd_1=sum_1/(len(y_lab_vd)*y_lab_vd.shape[1])
                result_validation=['Epoch:' , e, 'iteration:', iteration, 'Validation_loss:', loss_vd, 'Validation_sample_acc:', acc_vd_]
                result_validation_1=['Epoch:' , e, 'iteration:', iteration, 'Validation_loss:', loss_vd, 'Validation_constituent_acc:', acc_vd_1]
                validation_result_list.append(result_validation)
                validation_result_list_1.append(result_validation_1)
                
                print("Epoch: {}/{}".format(e, epochs),
                        "Iteration: {:d}".format(iteration),
                        "Validation loss: {:6f}".format(loss_vd),
                        "Validation sample acc: {:.6f}".format(acc_vd_),
                         "Validation constituent acc_1: {:.6f}".format(acc_vd_1))
            
            iteration += 1
        
        saver.save(sess,'file path')

In [ ]:
#Test code

test_loss = []
test_acc = []
test_acc_1=[]
iteration = 1

with graph.as_default():
    saver = tf.train.Saver()
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    file_list_tes= glob.glob('/file path/*.csv')
    arrays_tes = [np.loadtxt(file_name_tes, delimiter=',', dtype=np.float32) for file_name_tes in file_list_tes]
    data_tes = np.concatenate(arrays_tes, axis=0)
    sess.run(iterator_tes.initializer, feed_dict = {input_data_tes:data_tes})
    saver.restore(sess, 'file path')
    
    for i in  range(10):
        sum_ = 0
        sum_1 = 0
        X_te, y_te, y_te_, y_te_p, y_ind = sess.run([X_tes, y_tes, y_tes_, y_tes_p, y_tes_ind])
        X_te = np.reshape(X_te, (-1, 4001, 1))
        feed = {inputs_ : X_te, labels_1 : y_te, keep_prob_ : 1.0, learning_rate_ : learning_rate}
        batch_loss, logit = sess.run([cost, logits_2], feed_dict = feed)            
        y_lab = np.empty([batch_size_, 3])
        y_logit = np.empty([batch_size_, 3])
        ann_result = np.empty([batch_size_, 2])
        
        for i in range(batch_size_):
            if y_ind[i,0] == 2:
                y_lab[i]=np.argsort(y_te_[i])[-3:]
                y_lab[i]=np.sort(y_lab[i])
                y_logit[i]=np.argsort(logit[i])[-3:]
                y_logit[i]=np.sort(y_logit[i])
                b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                sum_1= sum_1+b1
                if b1 == 3:
                    sum_+=1
            elif y_ind[i,0] == 1:
                z=np.argsort(y_te_[i])[-2:]
                y_lab[i]=np.append(z, [-1])
                y_lab[i]=np.sort(y_lab[i])
                z_=np.argsort(logit[i])[-2:]
                y_logit[i]=np.append(z_, [-1])
                y_logit[i]=np.sort(y_logit[i])
                y_te_p[i] = np.sort(y_te_p[i])
                b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                sum_1= sum_1+b1
                if b1 == 3:
                    sum_+=1
            elif y_ind[i,0] == 0:
                z=np.argsort(y_te_[i])[-1:]
                y_lab[i]=np.append(z, [-1,-2])
                y_lab[i]=np.sort(y_lab[i])
                z_=np.argsort(logit[i])[-1:]
                y_logit[i]=np.append(z_, [-1,-2])
                y_logit[i]=np.sort(y_logit[i])
                y_te_p[i] = np.sort(y_te_p[i])
                b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                sum_1= sum_1+b1
                if b1 == 3:
                    sum_+=1
            else:
                print('Something Wrong happened!!!')
               
        '''
        The following code can also be used for test.
        The number of positive nodes on the output layer before the sigmoid-cross-entropy loss function can be used as being indicative of whether 
        the sample to be tested is unary, binary or ternary.
        k is the adjustable threshold parameter. Normally k=0, then almost the same test accuracy as the original code above can be obtained.
        If k>0, the test accuracy will improve especially for real-word-data test.
        '''        
        
        '''
        k=1
        if k!=0:
            sum_ = 0
            sum_1 = 0
            for i in range(batch_size_):
                if np.count_nonzero(logit[i] > 1.0) >= 3:
                    y_lab[i]=np.argsort(y_te_[i])[-3:]
                    y_lab[i]=np.sort(y_lab[i])
                    y_logit[i]=np.argsort(logit[i])[-3:]
                    y_logit[i]=np.sort(y_logit[i])                     
                    b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                    sum_1= sum_1+b1
                    if b1 == 3:
                        sum_+=1 
                elif np.count_nonzero(logit[i] > 1.0) == 2:
                    z=np.argsort(y_te_[i])[-2:]
                    y_lab[i]=np.append(z, [-1])
                    y_lab[i]=np.sort(y_lab[i])
                    z_=np.argsort(logit[i])[-2:]
                    y_logit[i]=np.append(z_, [-1])
                    y_logit[i]=np.sort(y_logit[i])
                    b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                    sum_1= sum_1+b1
                    if b1 == 3:
                        sum_+=1
                elif np.count_nonzero(logit[i] > 1.0) == 1:
                    z=np.argsort(y_te_[i])[-1:]
                    y_lab[i]=np.append(z, [-1,-2])
                    y_lab[i]=np.sort(y_lab[i])
                    z_=np.argsort(logit[i])[-1:]
                    y_logit[i]=np.append(z_, [-1,-2])
                    y_logit[i]=np.sort(y_logit[i])
                    b1 = len(list(set(y_lab[i]).intersection(y_logit[i])))
                    sum_1= sum_1+b1
                    if b1 == 3:
                        sum_+=1
                else:
                    print('Something Wrong happened!!!')
                    print(np.sort(logit[i])[::-1])
                    '''
                
        acc_te_=sum_/len(y_lab)            
        acc_te_1=sum_1/(len(y_lab)*y_lab.shape[1])
        test_loss.append(batch_loss)
        test_acc.append(acc_te_)
        test_acc_1.append(acc_te_1)
        
        print("Iteration: {:d}".format(iteration),
                  "Test loss: {:6f}".format(batch_loss),
                  "Test sample-based acc: {:.6f}".format(acc_te_),
                  "Test constituent-based acc_1: {:.6f}".format(acc_te_1))
        iteration += 1
    print("Average Test loss: {:.6f}".format(np.mean(test_loss)))
    print("Average Test sample_based accuracy: {:.6f}".format(np.mean(test_acc)))
    print("Average Test constituent_based accuracy: {:.6f}".format(np.mean(test_acc_1)))
    
####################################################################End of Test ##########################################################